## FLAN-T5教程：指南和微调 FLAN-T5 Tutorial: Guide and Fine-Tuning

In [1]:
# conda create -n flan_t5 python=3.10

In [2]:
# %%bash
# pip install nltk
# pip install datasets
# pip install 'transformers[torch]'
# pip install tokenizers
# pip install evaluate
# pip install rouge_score
# pip install sentencepiece
# pip install huggingface_hub

In [3]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

/Users/alvis/anaconda3/envs/flan_t5/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# # Load the tokenizer, model, and data collator
# MODEL_NAME = "google/flan-t5-base"
MODEL_PATH = "/Users/alvis/Study/models/flan-t5-base" # 从本地加载模型

tokenizer = T5Tokenizer.from_pretrained(MODEL_PATH)
model = T5ForConditionalGeneration.from_pretrained(MODEL_PATH)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
# Acquire the training data from Hugging Face
# DATA_NAME = "yahoo_answers_qa"
DATA_PATH = "/Users/alvis/Study/datasets/yahoo_answers_qa/nfL6.json"
# yahoo_answers_qa = load_dataset('json', DATA_PATH)
yahoo_answers_qa = load_dataset('json', data_files={'train': DATA_PATH})
print(yahoo_answers_qa)

DatasetDict({
    train: Dataset({
        features: ['main_category', 'question', 'nbestanswers', 'answer', 'id'],
        num_rows: 87362
    })
})


In [6]:
yahoo_answers_qa = yahoo_answers_qa["train"].train_test_split(test_size=0.3)
# Check the length of the data and its structure
yahoo_answers_qa

DatasetDict({
    train: Dataset({
        features: ['main_category', 'question', 'nbestanswers', 'answer', 'id'],
        num_rows: 61153
    })
    test: Dataset({
        features: ['main_category', 'question', 'nbestanswers', 'answer', 'id'],
        num_rows: 26209
    })
})

In [7]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question: "

# Define the preprocessing function

def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [prefix + doc for doc in examples["question"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)
  
   # The "labels" are the tokenized outputs:
   labels = tokenizer(text_target=examples["answer"], 
                      max_length=512,         
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

In [8]:
# Map the preprocessing function across our dataset
tokenized_dataset = yahoo_answers_qa.map(preprocess_function, batched=True)

Map:   0%|          | 0/61153 [00:00<?, ? examples/s]/Users/alvis/anaconda3/envs/flan_t5/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:287: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
Map: 100%|██████████| 26209/26209 [00:04<00:00, 5510.69 examples/s]


In [9]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

In [10]:
def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  
   return result

In [11]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 4
PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 3

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
#    evaluation_strategy="epoch",
   eval_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False,

#    use_mps_device=True,  # 关键参数
#    dataloader_pin_memory=False,  # 使用MPS时建议设置为False
)

In [12]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

/var/folders/z6/dz0yb7c95n9dtykytfy42j0r0000gn/T/ipykernel_71156/2420821063.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [13]:
trainer.train()

/Users/alvis/anaconda3/envs/flan_t5/lib/python3.10/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


RuntimeError: MPS backend out of memory (MPS allocated: 17.52 GiB, other allocations: 24.78 GiB, max allowed: 42.43 GiB). Tried to allocate 135.79 MiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).